# Install PySTAC Client

Import the PySTAC Client and connect the Client
to the STAC API endpoint.

PySTAC returns generator objects as a response that
can be iterated over. This abstracts away listing and pagination.

In [1]:
from pystac_client.client import Client
from pystac.item import Item
from pystac.collection import Collection
from pystac.link import Link
from pystac_client.asset_search import Asset

## Pre-Condition: connect to STAC endpoint

In [2]:
api = Client.open("http://127.0.0.1:8000")

Get basic information about the API from the Client

In [3]:
print(
    f"Title: {api.title}\n"
    f"Description: {api.description}\n"
    f"Catalog ID: {api.id}\n"
)

Title: test_api_title
Description: test api
Catalog ID: stac-fastapi



## 01: Basic free-text search, across all items in all collections

A collection must be specified, PySTAC supports all collection however
the stac-fastapi-elasticsearch encounters a 500 error. Will need to look into why.

Search query must be inside a query argument.

## 02: Same as `01`, with named parameter "q" instead of unnamed parameter

q search allows for case-insensitive search and partial match with wild-card.

PySTAC uses 'query' instead of 'q'

```py
query="AerChemMIP"
query="aerchemmip"
query="aerchemm*"
```

In [4]:
results = api.search(
    collections="cmip6",
    q="aerchemm*",
    max_items=1,
)
items = results.get_items()
item = next(items)
item.properties['activity_id']

['AerChemMIP']

## 03: Same as `02`: but checking search is case-insensitive

demonstrated in `02`

## 04: same as `01`, but specifying doctype

`api.search()` and `api.asset_search()` are item and asset level searches respectively.

In [5]:
items = api.search(collections='cmip6', max_items=1).get_items()

assets = api.asset_search(max_assets=1).get_assets()

asset: Asset = next(assets)
item: Item = next(items)

print(asset)
print(item)

<Asset href=http://esgf-data3.ceda.ac.uk/thredds/fileServer/esg_cmip6/CMIP6/CMIP/CAS/FGOALS-g3/historical/r1i1p1f1/fx/orog/gn/v20201202/orog_fx_FGOALS-g3_historical_r1i1p1f1_gn.nc>
<Item id=c2d94dd296525cc105cfa657b6b559c0>


## 05: like `03` but doctype is set to collection

There is no collection-level search

## 06: like `01`, but checking partial match works with a wild-card

Demonstated in `02`

## 07: like `01`, only search within the collections specified.

Must specify a collection to begin with.

Can not test multiple collections as of writing however the code does not show that capability
but is likely it can be abstracted in elasticsearch to work.

## 07b: like `07` only you can search collection objects and or collection IDs.

Collection ID can be retrieved from the object to search using a collection object
if ID is unknown. This will return a string.

In [6]:
collections = api.get_collections()
collection: Collection = next(collections)

items = api.search(collections=collection.id, max_items=1).get_items()
item = next(items)

print(f"item_id: {item.id} from collection: {item.collection_id}")

item_id: c2d94dd296525cc105cfa657b6b559c0 from collection: cmip6


## 08: facetted search, looking to match a given facet over all items

Faceted search can be done using the filter parameter.

In [7]:
results = api.search(
    collections=collection.id,
    max_items=10,
    filter={
        "eq": [{"property": "activity_id"}, "AerChemMIP"]
    }
).get_items()

item = next(results)
item.properties['activity_id']

['AerChemMIP']

## 09: faceted search, looked to match multiple facets over all items

In [8]:
results = api.search(
    collections=collection.id,
    max_items=10,
    method='GET',
    filter="activity_id = 'AerChemMIP' and source_id = 'UKESM1-0-LL'",
    filter_lang="cql2-text"
).get_items()

item = next(results)
item.properties

{'table_id': ['Amon'],
 'variable_long_name': ['Percentage Cloud Cover'],
 'height_units': ['m'],
 'dataset_version': ['20191022'],
 'data_node': ['esgf-data3.ceda.ac.uk'],
 'variant_label': ['r1i1p1f2'],
 'grid_label': ['gn'],
 'instance_id': ['CMIP6.AerChemMIP.MOHC.UKESM1-0-LL.histSST-piAer.r1i1p1f2.Amon.cl.gn.v20191022'],
 'grid': ['Native N96 grid; 192 x 144 longitude/latitude'],
 'nominal_resolution': ['250 km'],
 'realm': ['atmos'],
 'file_id': ['CMIP6.AerChemMIP.MOHC.UKESM1-0-LL.histSST-piAer.r1i1p1f2.Amon.cl.gn.v20191022.cl_Amon_UKESM1-0-LL_histSST-piAer_r1i1p1f2_gn_185001-189912.nc|esgf-data3.ceda.ac.uk',
  'CMIP6.AerChemMIP.MOHC.UKESM1-0-LL.histSST-piAer.r1i1p1f2.Amon.cl.gn.v20191022.cl_Amon_UKESM1-0-LL_histSST-piAer_r1i1p1f2_gn_190001-194912.nc|esgf-data3.ceda.ac.uk',
  'CMIP6.AerChemMIP.MOHC.UKESM1-0-LL.histSST-piAer.r1i1p1f2.Amon.cl.gn.v20191022.cl_Amon_UKESM1-0-LL_histSST-piAer_r1i1p1f2_gn_195001-199912.nc|esgf-data3.ceda.ac.uk',
  'CMIP6.AerChemMIP.MOHC.UKESM1-0-LL.histS

## 10: faceted search, looking to match multiple facets within multiple collections

Same as `09` but use the `mip_era` property to add a collections to the filter or
list the collection id in the `collections` parameter.

## 11: Global Asset-level search

In [9]:
assets = api.asset_search(
    filter={
        "eq": [{"property": "activity_id"}, "AerChemMIP"]
    }
).get_assets()

asset: Asset = next(assets)
asset.properties['activity_id']

'AerChemMIP'

## 12 Asset-level search within Collection

Can use global asset search and search by `mip_era` in the properties.

In [10]:
assets = api.asset_search(
    method='GET',
    filter="mip_era = 'CMIP6' and source_id = 'UKESM1-0-LL'",
    max_assets=1
).get_assets()
asset = next(assets)
asset.href

'http://esgf-data3.ceda.ac.uk/thredds/fileServer/esg_cmip6/CMIP6/GeoMIP/MOHC/UKESM1-0-LL/G6solar/r1i1p1f2/Omon/o2sat/gn/v20191031/o2sat_Omon_UKESM1-0-LL_G6solar_r1i1p1f2_gn_202001-204912.nc'

## 13: Asset-level search within Item

Can search within an item with the `items` parameter and providing an item id.

In [11]:
assets = api.asset_search(
    items=item.id,
    max_assets=1
).get_assets()
asset = next(assets)
print(
    f"item id: {item.id}\n"
    f"asset's item: {asset.item}"
)

item id: 6fdbe6aab785ccf046de62217cbfc5dc
asset's item: 6fdbe6aab785ccf046de62217cbfc5dc


## 14: Datetime Queries

### 14.1: Single date

Putting in a date transforms it to a datetime with range `T00:00:00Z` - `T23:59:59Z`.
By transforming into a range, the daterange of an item must be inside of
`2300-01-01T00:00:00Z` and `2300-01-01T23:59:59Z`

In [12]:
results = api.search(
    method="GET",
    datetime="2300-01-01",
)
results.matched()

0

### 14.2: Single datetime

In contrast to `14.1`, a single datetime is not a range so the query must be in the
daterange of the item.

In [13]:
results = api.search(
    method="GET",
    datetime="2300-01-01T00:00:00Z",
)
results.matched()

7979

### 14.3: Complete datetime range

Same as `14.1`, a full datetime with range requires the daterange of an item to be
inside the query.

In [14]:
results = api.search(
    method="GET",
    datetime="2300-01-01T00:00:00Z/2800-12-01T00:00:00.000Z",
)
results.matched()

62

### 14.4: Open ended datetime range


In [15]:
results = api.search(
    method="GET",
    datetime="2300-01-01T00:00:00Z/..",
)
results.matched()

1897

### 14.5: Open started datetime range


In [16]:
results = api.search(
    method="GET",
    datetime="../2800-12-01T00:00:00.000Z",
)
results.matched()

644107

### 14.6 POST search

In [17]:
results = api.search(
    method='POST',
    datetime="2300-01-01T00:00:00Z/2800-12-01T00:00:00.000Z",
)
results.matched()

62